In [162]:
import pandas as pd

# Load all data with pandas
data = pd.read_csv('/content/Tunneling_Induced_building_damage_dataset.txt', sep='\t')
data = data.drop(labels = 'Tot No. Simulations', axis=1)
data_columns = data.columns
data.head()

,E,Ft,Gft,Height,Length,Opening Rate,Distance,E_Soil,Soil_Piossons,Trough Width,...,Max H. Strain,Dam. Area (%),Strain (100%),Strain (99%),Strain (98%),Strain (97%),Strain (96%),Strain (95%),Local Avg.,Global Avg.
0,3653.786,0.607,0.027,7.203,20.554,22.119,16.729,219.134,0.153,10.240,...,0.000068,0.0,0.000076,0.000061,0.000056,0.000052,0.000050,0.000047,0.0,0.000990
1,5408.683,0.731,0.020,18.063,20.812,7.904,28.093,48.524,0.409,9.841,...,0.000013,0.0,0.000034,0.000018,0.000016,0.000016,0.000015,0.000015,0.0,0.000064
2,8876.708,0.908,0.019,12.925,28.951,15.831,29.218,203.529,0.318,17.677,...,0.000027,0.0,0.000030,0.000025,0.000023,0.000021,0.000020,0.000018,0.0,0.001330
3,4661.990,0.495,0.016,11.468,16.347,26.525,9.799,203.272,0.362,17.249,...,0.000009,0.0,0.000027,0.000017,0.000016,0.000015,0.000015,0.000014,0.0,0.000210
4,6558.350,0.953,0.027,10.088,23.745,10.630,6.822,173.406,0.124,29.213,...,0.000009,0.0,0.000017,0.000012,0.000011,0.000011,0.000010,0.000010,0.0,0.000124


In [163]:
from sklearn.model_selection import train_test_split

X = data.iloc[:,:15]
Y = data.iloc[:,15:]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=42)

Y_train = Y_train.to_numpy()
Y_test = Y_test.to_numpy()

In [164]:
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler, Normalizer, PowerTransformer, RobustScaler, StandardScaler

# Transform data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Transform data back to original
# data_s = scaler.inverse_transform(data_s)
# pd.DataFrame(data_s, columns=data_columns).head()

In [165]:
# pd.DataFrame(X_train, columns=data_columns[:15]).head()
pd.DataFrame(X_test, columns=data_columns[:15]).head()

,E,Ft,Gft,Height,Length,Opening Rate,Distance,E_Soil,Soil_Piossons,Trough Width,Friction,VL,Depth,Diameter,Fc
0,2.303142,2.354680,1.608259,0.704272,0.636719,0.846019,-1.517808,0.030140,0.613609,0.870619,-1.251578,0.826008,-0.014029,-1.982249,2.830373
1,-0.174519,0.418581,1.608259,-0.643444,0.984876,-0.368778,1.704222,-1.212019,0.613609,0.336558,-0.922314,-1.382548,-0.469546,0.467852,-0.035664
2,-1.390041,-1.801671,-1.938524,-1.274412,1.295150,-0.415168,-1.230843,0.743971,0.112446,1.525625,1.182389,1.118818,1.598753,-1.111994,-1.063810
3,0.264880,-0.053403,0.528804,-0.298782,-0.860649,-1.286536,-0.325198,-1.631440,0.318281,0.593457,1.931302,0.193804,1.300215,-1.857927,-0.591375
4,-1.046914,-1.238180,-0.859068,0.114770,0.221930,0.144554,-0.282200,0.900592,-0.737743,-0.156480,-0.702805,-0.733707,0.804316,1.627235,-0.786554


In [166]:
print((X_train.shape))
print((X_test.shape))
print((Y_train.shape))
print((Y_test.shape))

(487, 15)
(487, 15)
(487, 16)
(487, 16)


In [167]:
# GRNN class by Izonin
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin

class GRNN(BaseEstimator, RegressorMixin):
    def __init__(self, name = "GRNN", sigma = 0.1):
        self.name = name
        self.sigma = 2 * np.power(sigma, 2)

    def predict(self, instance_X, X_train_scaled, Y_train):
        gausian_distances = np.exp(-np.power(np.sqrt((np.square(X_train_scaled-instance_X).sum(axis=1))),2) / self.sigma)
        gausian_distances_sum = gausian_distances.sum()
        gausian_distances_sum = max(gausian_distances_sum, 1e-07)
        return np.multiply(gausian_distances, Y_train).sum() / gausian_distances_sum

In [168]:
from scipy.optimize import differential_evolution
from sklearn.metrics import max_error, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, median_absolute_error, r2_score
import time

results = {}
# cost function to optimize
def f(params, X_train, Y_train, X_test, Y_test):
    s, = params  # Unpack the parameters
    grnn = GRNN(sigma=s)
    predictions = np.array([grnn.predict(i, X_train, Y_train) for i in X_test])

    return mean_squared_error(Y_test, predictions) # USE MSE


for i in range(Y_train.shape[1]):
    start_time = time.time()

    res = differential_evolution(f, bounds=[(0.001, 10)], args=(X_train, Y_train[:, i], X_test, Y_test[:, i]))
    s = res["x"][0]

    grnn = GRNN(sigma=s)
    predictions = np.apply_along_axis(lambda x: grnn.predict(x, X_train, Y_train[:, i]), axis=1, arr=X_test)

    exp_time = time.time() - start_time


    MaxError = max_error                    (Y_test[:,i].ravel(), predictions)
    MAE = mean_absolute_error               (Y_test[:,i].ravel(), predictions)
    MSE = mean_squared_error                (Y_test[:,i].ravel(), predictions)
    MedError = median_absolute_error        (Y_test[:,i].ravel(), predictions)
    RMSE = mean_squared_error               (Y_test[:,i].ravel(), predictions, squared=False)
    MAPE = mean_absolute_percentage_error   (Y_test[:,i].ravel(), predictions)
    R2 = r2_score                           (Y_test[:,i].ravel(), predictions)

    results.update({
        f'test_{i+1}':
            {
                'time': exp_time,
                'sigma': s,
                'y_true': Y_test[:,i].ravel(),
                'y_pred': predictions,
                'MaxError' : MaxError,
                'MAE' : MAE,
                'MSE' : MSE,
                'MedError' : MedError,
                'RMSE' : RMSE,
                'MAPE' : MAPE,
                'R2' : R2
            }
    })

exp_result = pd.DataFrame(results)

In [169]:
exp_result

,test_1,test_2,test_3,test_4,test_5,test_6,test_7,test_8,test_9,test_10,test_11,test_12,test_13,test_14,test_15,test_16
time,2.506138,5.443438,4.536958,2.358784,3.505298,4.862442,2.002245,3.302499,2.792583,3.289862,2.84735,3.202566,2.391552,3.404462,3.622825,2.389453
sigma,0.914873,1.053887,0.886584,0.909178,0.932327,0.820722,1.063032,0.901983,1.046418,1.058954,1.018495,0.87446,0.892295,0.911991,0.946167,0.894119
y_true,"[0.00031, 0.00028, 0.00032, 0.00017, 0.00178, ...","[0.0, 0.0, 0.0, 0.0, 4.81, 2.15, 0.0, 0.0, 0.0...","[0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 0.0, ...","[0.00011, 0.00068, 0.00052, 0.00027, 0.00241, ...","[9e-05, 0.00066, 0.00051, 0.00027, 0.00236, 0....","[2.5e-05, 6.5e-05, 0.0001, 5e-06, 0.000113, 6....","[2.6e-05, 6.4e-05, 5.8e-05, 9e-06, 0.000943, 0...","[0.0, 0.0, 0.0, 0.0, 0.71, 0.79, 0.0, 0.0, 0.0...","[3.6e-05, 6.7e-05, 6e-05, 2.1e-05, 0.027513, 0...","[2.5e-05, 5.8e-05, 5.4e-05, 9e-06, 9.5e-05, 7e...","[2.3e-05, 5.4e-05, 5.2e-05, 8e-06, 8.2e-05, 5....","[2.1e-05, 5.2e-05, 4.9e-05, 8e-06, 7.4e-05, 5....","[1.9e-05, 5e-05, 4.7e-05, 8e-06, 6.6e-05, 4.7e...","[1.8e-05, 4.8e-05, 4.5e-05, 8e-06, 5.9e-05, 4....","[0.0, 0.0, 0.0, 0.0, 2.90745, 2.07435, 0.0, 0....","[6.2e-05, 0.000367, 0.000297, 0.000137, 0.0014..."
y_pred,"[0.0013419603476474713, 0.00094700153030415, 0...","[1.372385295668483, 5.51736998348354, 2.755758...","[0.9371112947403712, 2.5890177674119323, 2.758...","[0.0013073229934410272, 0.002328438805997238, ...","[0.0012618598825837575, 0.00225135138251307, 0...","[8.506506039597964e-05, 0.0003911015900129844,...","[0.0002944886093364812, 0.001013041417749621, ...","[0.38366521035183243, 1.8270648412465078, 1.08...","[0.009384662301565977, 0.030490581588470556, 0...","[0.0002951645465848251, 0.0024913206598935317,...","[8.682197304329633e-05, 0.0007445324000811859,...","[3.1005585544639975e-05, 0.000225176418240887,...","[2.8230437330851138e-05, 0.0001116224085625285...","[2.6569325564887667e-05, 7.508633333965153e-05...","[1.1350936248599977, 4.058684407504151, 2.7342...","[0.0007256004892325025, 0.0014762412286799297,..."
MaxError,0.006386,38.765561,14.510381,0.006462,0.00535,0.002339,0.007475,10.080395,0.202844,0.051683,0.007603,0.004511,0.003899,0.00309,16.517266,0.003839
MAE,0.000735,2.741247,1.899708,0.000782,0.000754,0.000212,0.000585,1.079462,0.014679,0.002129,0.000766,0.000351,0.000195,0.000117,2.071064,0.00052
MSE,0.000001,18.577161,7.776449,0.000001,0.000001,0.0,0.000001,2.566018,0.000516,0.000017,0.000001,0.0,0.0,0.0,8.660026,0.000001
MedError,0.000538,2.037976,1.332425,0.000628,0.000615,0.000136,0.000423,0.792843,0.011365,0.001328,0.000505,0.000155,0.000069,0.000039,1.508744,0.000403
RMSE,0.001077,4.310123,2.788629,0.001037,0.000985,0.000355,0.000894,1.601879,0.022721,0.004166,0.001219,0.000656,0.000432,0.000303,2.942792,0.000714
MAPE,1.738387,5057071916903969.0,3129614126872145.5,1.472784,1.561973,3.007097,9.520475,1583419823100763.75,148.754242,24.106478,10.569442,4.700152,2.726439,1.735604,3965118281270101.0,1.393555


In [173]:
exp_result.to_excel('MinMaxScaler.xlsx')